In [1]:
import pandas as pd
import random 
import itertools
import json
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, distance_matrix_calc, run_min_variance, run_clustering_model, test_for_silhouette_score

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)

joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level

In [3]:
joined_df_weekly

,AAPL,ADBE,AMD,AMZN,ASML,AVGO,COST,CRESY,CSCO,ERIC,...,SAND-USD,SHIB-USD,SNX-USD,SOL-USD,STX-USD,UNI-USD,XLM-USD,XRP-USD,XTZ-USD,ZEC-USD
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-09,169.209167,510.700012,132.000000,162.554001,733.287048,57.696838,513.291748,3.853515,55.273987,9.952560,...,4.848104,0.000029,5.394906,136.402817,2.092521,0.000249,0.256958,0.763074,4.256260,139.588547
2022-01-16,170.093674,520.599976,136.880005,162.138000,722.066345,55.574028,481.518463,4.135884,55.481949,10.241040,...,4.901487,0.000031,5.238216,146.159027,2.092526,0.000259,0.266340,0.774297,4.285115,142.475052
2022-01-23,159.617004,499.910004,118.809998,142.643005,673.768799,49.690193,461.051147,4.011310,51.250267,9.925517,...,3.336782,0.000022,4.173945,111.966057,1.672486,0.000219,0.202986,0.635738,3.331553,102.061218
2022-01-30,167.400803,518.159973,105.239998,143.977997,625.510132,52.194126,471.409302,4.185715,50.282776,10.881105,...,3.489832,0.000021,4.566989,91.312897,1.462516,0.000113,0.198722,0.612512,3.026571,91.409630
2022-02-06,169.641220,513.539978,123.599998,157.639496,633.113708,54.995335,498.336395,4.227241,49.866840,11.214661,...,4.005184,0.000022,5.522647,112.174248,1.539816,0.000083,0.208888,0.655877,4.014410,107.115463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-03,190.098663,612.469971,121.389999,147.029999,686.687622,91.068413,578.967834,8.108377,46.315083,4.737260,...,0.415088,0.000008,3.420732,59.803997,0.002968,0.001386,0.119794,0.612915,0.842495,29.772724
2023-12-10,194.541977,610.010010,128.919998,147.419998,690.883850,92.468719,593.076660,8.683099,46.229092,5.341214,...,0.494491,0.000010,3.865750,75.002434,0.004156,0.000221,0.132325,0.673138,0.944927,33.418129
2023-12-17,196.390900,584.679993,139.149994,149.970001,746.963684,110.627556,639.724304,9.351029,47.652836,5.652628,...,0.496234,0.000010,4.023122,72.368988,0.005516,0.000210,0.122939,0.615492,0.916226,29.495764


In [4]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [5]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [6]:
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)

In [7]:
#Reassemble the results of the generation
# min_var_portfolios = dict()
# for i in range(1,1000):
#     with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
#         port = json.load(f)
#         min_var_portfolios.update(port)
    

In [8]:
# dtw_matrix = dtw_matrix_calc(joined_df)

#labels_kmeans, t, centroids_kmeans = run_clustering_model(joined_df, n_clus=3, model_name='kmeans')


# labels_kshape = run_clustering_model(joined_df, n_clus=3, model_name='kshape')

# labels_ahc = run_clustering_model(joined_df, n_clus=3, model_name='ahc', linkage='single')

In [9]:
n_clusters_list = [2,3,4,5,6,7]

#kshape_inertia = test_for_silhouette_score(joined_df_weekly, n_clusters_list, method='kshape', return_mode='geometric')
#silhouette_df_ahc = test_for_silhouette_score(joined_df, n_clusters_list, method='ahc', linkage_list=['single', 'average', 'complete'])
silhouette_df_kmeans = test_for_silhouette_score(joined_df_weekly, n_clusters_list, method='kmeans', return_mode='geometric', soft_dtw=True, gamma=0.5)

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


ValueError: Input X contains NaN.

In [ ]:
silhouette_df_kmeans

,clusters,inertia_score,method
0,2,0.367099,kshape
1,3,0.261936,kshape
2,4,0.220034,kshape
3,5,0.199970,kshape
4,6,0.198428,kshape
5,7,0.181389,kshape


In [ ]:
# silhouette_df_kmeans.to_csv('silhouette_df_kmeans')
# silhouette_df_kshape.to_csv('silhouette_df_kshape')
# silhouette_df_ahc.to_csv('silhouette_df_ahc')

In [ ]:
plt.figure(figsize=(8, 5))

for linkage_type in silhouette_df_ahc['linkage'].unique():
    subset = silhouette_df_ahc[silhouette_df_ahc['linkage'] == linkage_type]
    plt.plot(subset['clusters'], subset['silhouette_score'],
             marker='o',
             label=f'Linkage: {linkage_type}')  # Use linkage as the label

plt.xlabel("Number of clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores by Linkage Method")
plt.grid(True)
plt.legend()
plt.show()

NameError: name 'silhouette_df_ahc' is not defined

<Figure size 800x500 with 0 Axes>

In [ ]:
plt.figure(figsize=(8, 5))


plt.plot(silhouette_df_kshape['clusters'], silhouette_df_kshape['silhouette_score'],
             marker='o')  # Use linkage as the label

plt.xlabel("Number of clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores per number of clusters")
plt.grid(True)
plt.legend()
plt.show()

NameError: name 'silhouette_df_kshape' is not defined

<Figure size 800x500 with 0 Axes>